# Context Engineering: LLMs for LF

## Objective
Generate Lingua Franca (LF) code from natural language prompt.

## Context
Large Language Models (LLMs) are a powerful tool for generating code. However, LLMs are often trained on standard programming languages like C/C++, Java, Python, and JavaScript. This makes it difficult to generate code in Domain-specific languages like Lingua Franca.

This project shows how to curate context to guide the LLM.  We demonstrate the technique by showing how to generate code that uses the Lingua Franca coordination language together with C and Python.

## 1 Environment Setup:
- Virtual environment installation.
- Necessary libraries installation.
- Necessary imports.
- OPEN_AI_API_KEY loading.

In [1]:
# Installing Required Packages
!python -m pip install --upgrade pip
!python -m pip install python-dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.6 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
# Importing Libraries
import os
from dotenv import load_dotenv
from openai import OpenAI

A `.env` file is used to store sensitive information that you don't want to hard-code into your source code.


For this project `OPENAI_API_KEY` is stored in `.env` file as:


`OPENAI_API_KEY='your_openai_api_key'`



In [3]:
# Loading OPENAI_API_KEY from the .env file
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

In [6]:
"""
A function to estimate the probability that at least one of the top k samples is correct, 
given that there are c-correct samples in total out of n-generated samples.
"""
import numpy as np
def pass_at_k(n, c, k): 
  """ 
  :param n: total number of samples 
  :param c: number of correct samples 
  :param k: k in pass@$k$ 
  """ 
  if n - c < k: 
    return 1.0 
  return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

### 2 Monte Carlo Estimation of π


#### 2.1 Prompting For regular Python code
We start by prompting OpenAI's `gpt-4o` without any additional context.

The generated code will be stored under `LFGPT/Python/pi`.

We start by creating the target directory.

In [7]:
# Creating the target directory
!mkdir -p LFGPT/Python/pi

In [29]:
"""
  A function to ganarate code.
  Inputs:
    file_name: name of the file to create
    prompt: prompt to be used for generating code
  Returns:
    No value is returned.

"""

def code_generator(file_name, prompt):
  # Initializes an OpenAI client for interacting with the OpenAI API.
  client = OpenAI()


  MODEL = "gpt-4o"

  fout = open(file_name, 'w')

  completion = client.chat.completions.create(
    model=MODEL,
    temperature=0.8,
    messages=[
      {"role": "system", "content": prompt}
    ]
  )

  print(completion.choices[0].message.content, file=fout)
  fout.close()

  print(file_name, " sucessfully generated.")



In [41]:
client = OpenAI()


MODEL = "gpt-4o"

fout = open(file_name, 'w')

completion = client.chat.completions.create(
model=MODEL,
temperature=0.8,
messages=[
    {"role": "system", "content": prompt}
]
)


In [43]:

# Defining the prompt for a sequential execution
prompt = f""" 
  Provide a python code to estimate PI through Monte Carlo Method with 10 million samples.
  Provide code only, without any comment or code fences.
  """
# File name suffix
file_name = "LFGPT/Python/pi/MonteCarlo"

# Defining run iterartor
run_iterator = 0


In [44]:
run_iterator += 1
# Name of the file to create
f_name = f'{file_name}_{run_iterator}.py'
# Generate code
code_generator(f_name, prompt)

import subprocess
# Execute the code

result = subprocess.run(['python3', f_name], capture_output=True, text=True)
print("results: ", result.stdout)
print("errors: ", result.stderr)

LFGPT/Python/pi/MonteCarlo_1.py  sucessfully generated.
results:  3.1409356

errors:  


#### 2.2 Prompting For parallel Python code

In [45]:

# Defining the prompt
prompt = f""" 
  Provide a python code to estimate PI through Monte Carlo Method with 10 million samples.
  The code should run in parallel and prints the number of processeses used.
  Provide code only, without any comment or code fences.
  """
# File name suffix
file_name = "LFGPT/Python/pi/ParallelMonteCarlo"

# Defining run iterartor
run_iterator = 0


In [58]:
run_iterator += 1
# Name of the file to create
f_name = f'{file_name}_{run_iterator}.py'
# Call code generator
code_generator(f_name, prompt)
# Run the generate file
import subprocess

result = subprocess.run(['python', f_name], capture_output=True, text=True)
print("results: ", result.stdout)
print("errors: ", result.stderr)

LFGPT/Python/pi/ParallelMonteCarlo_5.py  sucessfully generated.
results:  Estimated PI: 3.141682
Number of processes used: 12

errors:  


In [66]:
!python LFGPT/Python/pi/ParallelMonteCarlo_1.py


Estimated PI: 3.1416552
Number of processes used: 12


#### 2.3 Prompting For sequential LF code


##### 2.3.1 Prompting Without context
We start by prompting OpenAI's `gpt-4o` without any additional context.

The generated code will be stored under `LFGPT/without_context/pi`.

We start by creating the target directory.

In [24]:
# Creating the target directory
!mkdir -p LFGPT/without_context/pi

In [67]:

# Defining the prompt
prompt = f""" 
  Provide a Lingua Franca code for 'target Python'. 
  Create a main reactor named `MonteCarlo` that:
  - has no outputs.
  - has a parameter named `num_samples`, with a default value of 10 millions.
  - estimates PI through Monte Carlo Method with `num_samples`.
  - prints the estimated value of PI to the console.

  Provide code only, without any comment or code fences.
  """
# File name suffix
file_name = "LFGPT/without_context/pi/MonteCarlo"

# Defining run iterartor
run_iterator = 0


In [83]:
import shutil
import subprocess

def run_xp(file_name, prompt, generator, samples):
    errors = " ----------  Compile errors ------------\n"
    exec_errors = " ----------  Exeecution errors ------------\n"

    xp = {
        "compile_sucess": 0,
        "compile_error": 0,
        "run_sucess": 0,
        "run_error": 0
    }

    for run_iterator in range(samples):
        # Name of the file to create
        f_name = f'{file_name}_{run_iterator}.lf'
        # Generate code 
        generator(f_name, prompt)

        # Copy the generated file into a file that matches the main reactor name.
        shutil.copy(f_name, f'{file_name}.lf')

        
        # Compile the generated LF file
        result = subprocess.run(['lfc', f'{file_name}.lf'], capture_output=True, text=True)
        if result.returncode == 0:
            xp['compile_sucess'] += 1
            # Run the generated Python code
            executable = 'src-gen/' + file_name + "/" +file_name.split('/')[-1] + '.py'
            exec_result = subprocess.run(['python3', executable], capture_output=True, text=True)
            if exec_result.returncode == 0:
                xp['run_sucess'] += 1
            else:
                xp['run_error'] += 1
                exec_errors += exec_result.stderr
        else:
            xp['compile_error'] += 1
            errors += result.stderr
            
    print(f'Compile success: {xp["compile_sucess"]}')

    print(f'Run success: {xp["run_sucess"]}')


    print(errors, file=open(file_name + "_compile_errors.txt", 'w', encoding='utf-8'))
    print(exec_errors, file=open(file_name + "_exec_errors.txt", 'w', encoding='utf-8'))


In [86]:
run_xp(file_name, prompt, code_generator, 100)

LFGPT/without_context/pi/MonteCarlo_0.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_1.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_2.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_3.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_4.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_5.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_6.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_7.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_8.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_9.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_10.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_11.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_12.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_13.lf  sucessfully generated.
LFGPT/without_context/pi/MonteCarlo_14.lf  sucessfully generated.
LFGPT/without_contex

##### 2.3.2 Prompting with RAG





In [88]:
# Importing packages
import os 
from llama_index.core import (
    load_index_from_storage,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex
    
)

# Setting codebase directory
# contains examples directory from https://github.com/lf-lang/playground-lingua-franca
DIR = "./python_codebase"

# Setting storage directory
PERSIST_DIR = "./index_python_codebase"

In [89]:
# If `VectorStoreIndex` exists load it from disk.
# Else, Build it from code_base and save it to disk, for future use.

def load_storage_context():
    # if 'PERSIST_DIR' exists, load indexes from persisted data
    # else load create indexes from code base dierctory, 'data'.
    if (os.path.exists(PERSIST_DIR)):
        # load the existing index
        print("Loading persisted indexes ...")
        # rebuild storage context
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
        return storage_context      
    else:
        print("Creating indexes from scratch ...")
        # load documents from directory
        documents = SimpleDirectoryReader(DIR, recursive=True).load_data()

        # build index with embedding model "gpt-4o"
        index = VectorStoreIndex.from_documents(documents, model="gpt-4o")

        # save indexes on disk
        index.storage_context.persist(persist_dir=PERSIST_DIR) 
        return index.storage_context

# Loading `VectorStoreIndex` (storage context) from disk
storage_context = load_storage_context()


Creating indexes from scratch ...


In [90]:
# Creating the target directory
!mkdir -p LFGPT/rag/pi

In [91]:
from llama_index.llms.openai import OpenAI

# Define LLM to use for Code geneartion
MODEL = "gpt-4o"

# Define the Top K retrived documents (LF files)
TOP_K = 10

# Instatntiate the LLM
llm = OpenAI(temperature=0.8, model=MODEL)

# Instantiate a ServiceContext using the OpenAI_API_key
index = load_index_from_storage(storage_context, llm=llm)        
 
# Configure retriever
query_engine = index.as_query_engine(similarity_top_k=TOP_K)



# Defining run iterartor
run_iterator = 0

# File name suffix
file_name = "LFGPT/rag/pi/MonteCarlo"

In [94]:

def rag(file_name, prompt):    
    response = query_engine.query(prompt)
    files = '/**\n'
    for node in response.source_nodes:
        files += f'* {node.metadata["file_path"]}\n'
    files += '*/\n'

    fout = open(file_name, 'w')

    print(files+response.response, file=fout)
    fout.close()

    print(file_name)




### Prompt engineering
- imports are not embedded in a preamble. Add:
```embed imports in a preamble.```
- The generated code has no `reaction`. The Python code needs to be inside a reaction. Explicitly instruct the LLM to use a reaction.

In [111]:
# Defining the prompt
prompt = f""" 
  Provide a Lingua Franca code for 'target Python'. 
  imports the random module.
  Create a main reactor named `MonteCarlo` that:
  - has no outputs.
  - has a reaction that triggers at startup.
  - the reaction estimates PI through Monte Carlo Method with  10 million samples.
  - prints the estimated value of PI to the console.
  - embed imports in a preamble.


  Provide code only, without any comment or code fences.
  """

In [113]:
run_xp(file_name, prompt, rag, 100)

LFGPT/rag/pi/MonteCarlo_0.lf
LFGPT/rag/pi/MonteCarlo_1.lf
LFGPT/rag/pi/MonteCarlo_2.lf
LFGPT/rag/pi/MonteCarlo_3.lf
LFGPT/rag/pi/MonteCarlo_4.lf
LFGPT/rag/pi/MonteCarlo_5.lf
LFGPT/rag/pi/MonteCarlo_6.lf
LFGPT/rag/pi/MonteCarlo_7.lf
LFGPT/rag/pi/MonteCarlo_8.lf
LFGPT/rag/pi/MonteCarlo_9.lf
LFGPT/rag/pi/MonteCarlo_10.lf
LFGPT/rag/pi/MonteCarlo_11.lf
LFGPT/rag/pi/MonteCarlo_12.lf
LFGPT/rag/pi/MonteCarlo_13.lf
LFGPT/rag/pi/MonteCarlo_14.lf
LFGPT/rag/pi/MonteCarlo_15.lf
LFGPT/rag/pi/MonteCarlo_16.lf
LFGPT/rag/pi/MonteCarlo_17.lf
LFGPT/rag/pi/MonteCarlo_18.lf
LFGPT/rag/pi/MonteCarlo_19.lf
LFGPT/rag/pi/MonteCarlo_20.lf
LFGPT/rag/pi/MonteCarlo_21.lf
LFGPT/rag/pi/MonteCarlo_22.lf
LFGPT/rag/pi/MonteCarlo_23.lf
LFGPT/rag/pi/MonteCarlo_24.lf
LFGPT/rag/pi/MonteCarlo_25.lf
LFGPT/rag/pi/MonteCarlo_26.lf
LFGPT/rag/pi/MonteCarlo_27.lf
LFGPT/rag/pi/MonteCarlo_28.lf
LFGPT/rag/pi/MonteCarlo_29.lf
LFGPT/rag/pi/MonteCarlo_30.lf
LFGPT/rag/pi/MonteCarlo_31.lf
LFGPT/rag/pi/MonteCarlo_32.lf
LFGPT/rag/pi/MonteCa

### Results
On 100 experiments (generated LF files), the AI was able to predict the correct LF code 93 times out of 100.

Compile success: 93

Run success: 93

The `pass@k` metric is defined as the probability that at least one of the top k-generated code samples for a problem passes the unit tests. This approach is inspired by the practices of human developers, who judge the correctness of code based on whether it passes a set of unit tests.

The folling is `pass@k` computation:

In [117]:
for K in [1,2,5,10]:
    print(f"pass@{K}: ", round(pass_at_k(100, 93, K),4))

pass@1:  0.93
pass@2:  0.9958
pass@5:  1.0
pass@10:  1.0


In [ ]:
!lfc LFGPT/rag/pi/MonteCarlo.lf
!python src-gen/LFGPT/rag/pi/MonteCarlo/MonteCarlo.py

#### Prompting For parallel LF code (Python target)

In [157]:
# Defining the prompt
prompt = f""" 
  Provide a Lingua Franca code for 'target Python'. 
  Use a preamble to include imports.
  Create a reactor named `MonteCarlo` that:
  - has one output port named `out` of type `float`.
  - has a reaction that triggers at startup.
  - the reaction estimates PI through Monte Carlo Method with forth of 10 million samples.
  - sets the output to the estimated value of PI.

  Create a federated reactor named `ParallelMonteCarlo` that:
  - instantiates 4 reactors named `MonteCarlo` in parallel.
  - sum the outputs of the 4 reactors and prints the result.

  Provide code only, without any comment or code fences.
  """

# File name suffix
file_name = "LFGPT/rag/pi/ParallelMonteCarlo"




In [161]:

run_xp(file_name, prompt, rag, 100)

LFGPT/rag/pi/ParallelMonteCarlo_0.lf
LFGPT/rag/pi/ParallelMonteCarlo_1.lf
LFGPT/rag/pi/ParallelMonteCarlo_2.lf
LFGPT/rag/pi/ParallelMonteCarlo_3.lf
LFGPT/rag/pi/ParallelMonteCarlo_4.lf
LFGPT/rag/pi/ParallelMonteCarlo_5.lf
LFGPT/rag/pi/ParallelMonteCarlo_6.lf
LFGPT/rag/pi/ParallelMonteCarlo_7.lf
LFGPT/rag/pi/ParallelMonteCarlo_8.lf
LFGPT/rag/pi/ParallelMonteCarlo_9.lf
LFGPT/rag/pi/ParallelMonteCarlo_10.lf
LFGPT/rag/pi/ParallelMonteCarlo_11.lf
LFGPT/rag/pi/ParallelMonteCarlo_12.lf
LFGPT/rag/pi/ParallelMonteCarlo_13.lf
LFGPT/rag/pi/ParallelMonteCarlo_14.lf
LFGPT/rag/pi/ParallelMonteCarlo_15.lf
LFGPT/rag/pi/ParallelMonteCarlo_16.lf
LFGPT/rag/pi/ParallelMonteCarlo_17.lf
LFGPT/rag/pi/ParallelMonteCarlo_18.lf
LFGPT/rag/pi/ParallelMonteCarlo_19.lf
LFGPT/rag/pi/ParallelMonteCarlo_20.lf
LFGPT/rag/pi/ParallelMonteCarlo_21.lf
LFGPT/rag/pi/ParallelMonteCarlo_22.lf
LFGPT/rag/pi/ParallelMonteCarlo_23.lf
LFGPT/rag/pi/ParallelMonteCarlo_24.lf
LFGPT/rag/pi/ParallelMonteCarlo_25.lf
LFGPT/rag/pi/ParallelM

In [167]:
!lfc LFGPT/rag/pi/ParallelMonteCarlo.lf
!python src-gen/LFGPT/rag/pi/ParallelMonteCarlo/ParallelMonteCarlo.py

lfc: warning: File 'ParallelMonteCarlo.lf' is not located in an 'src' directory.
lfc: warning: Adopting the current working directory as the package root.
lfc: info: ##### Generating code for federate federate__m1 in directory /home/moez/AccountableAI/LFGPT/lf-gpt/./fed-gen/ParallelMonteCarlo/src
lfc: info: ##### Generating code for federate federate__m2 in directory /home/moez/AccountableAI/LFGPT/lf-gpt/./fed-gen/ParallelMonteCarlo/src
lfc: info: ##### Generating code for federate federate__m3 in directory /home/moez/AccountableAI/LFGPT/lf-gpt/./fed-gen/ParallelMonteCarlo/src
lfc: info: ##### Generating code for federate federate__m4 in directory /home/moez/AccountableAI/LFGPT/lf-gpt/./fed-gen/ParallelMonteCarlo/src
lfc: info: ******** Using 1 threads to compile the program.
lfc: info: Generating code for: file:/home/moez/AccountableAI/LFGPT/lf-gpt/./fed-gen/ParallelMonteCarlo/src/federate__m1.lf
lfc: info: Generation mode: STANDALONE
lfc: info: Generating sources into: /home/moez/Acc

In [154]:
# response = query_engine.query("does Lingua Franca support parallelism?")
# print(response.response)
# response = query_engine.query("How can I exploit parallelism with lingua franca?")
# print(response.response)
response = query_engine.query("Provide a code for a minimal federated execution with lingua franca?")
print(response.response)

import Py_Federated from '../assets/code/py/src/Federated.lf';


### Common errors:
1- Main reactor cannot have outputs.
    generated code: `output pi_estimate`
    prompt solving: `has no outputs.`

2-  No viable alternative at input.
    generated code: `parameter num_samples = 10000000`
    prompt solving: remove `has parameter` from prompt.

3-  Syntactic error.
    generated code: `pimport random` without preamble.
    caused by: `import all the necessary libraries.` from prompt.
    prompt solving: not found yet.

    